In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import random
import os

In [3]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)

In [4]:
from poorman_transformer.data                 import TokenManager
from poorman_transformer.modeling.transformer import Transformer
from poorman_transformer.utils                import init_logger, MetaLog, save_checkpoint, load_checkpoint, set_seed

In [5]:
input_file_path = "input.txt"
with open(input_file_path, 'r') as fh:
    data = fh.read()
token_lib = sorted(list(set(data)))
token_manager = TokenManager(token_lib)

In [6]:
print(''.join(token_lib))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
data[234:234+32]

" to the people.\n\nAll:\nWe know't,"

In [8]:
# Define model
token_lib_size = len(token_lib)
embd_size      = 64
num_blocks     = 4
head_size      = 64 // 4
context_length = 32
num_heads      = embd_size // head_size

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Transformer(token_lib_size, embd_size, context_length, num_blocks, num_heads).to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')


0.210497 M parameters


In [43]:
# [[[ USER INPUT ]]]
timestamp_prev = "2023_0822_1233_05"
epoch          = 240

drc_chkpt = "chkpts"
fl_chkpt_prev   = None if timestamp_prev is None else f"{timestamp_prev}.epoch_{epoch}.chkpt"
path_chkpt_prev = None if fl_chkpt_prev is None else os.path.join(drc_chkpt, fl_chkpt_prev)

chkpt = torch.load(path_chkpt_prev)

# Load weights...
model.module.load_state_dict(chkpt['model_state_dict']) if hasattr(model, 'module') else \
        model.load_state_dict(chkpt['model_state_dict'])

<All keys matched successfully>

In [45]:
input = "X-ray"
# input = input[:32]
input_encoded = token_manager.encode(input[-context_length:])
input_encoded = torch.tensor(input_encoded)[None,].to(device)

# input_encoded = torch.zeros((1, 1), dtype = torch.long).to(device)

print(f"___/ Input \___")
print(input)
print("")

for _ in range(500):
    next_token_encoded = model.generate_one(input_encoded[:,-context_length:])
    input_encoded = torch.cat([input_encoded, next_token_encoded], dim = 1)


output = token_manager.decode(input_encoded.cpu().tolist()[0])
print(f"___/ Output \___")
print(f"{output}")


___/ Input \___
X-ray

___/ Output \___
X-rays there is murderer!

GREEN:
Speaks, sir, and can become, and tollow,
I have mets the senated for his cowardly heart to my lord; thou shalt thou canst
What more special as in my father?

Bore Servingman:
Were it me are article; and for talk with that may letters,
into them told, for my dates, where is it murderate,
Or keep awhalesome sack here
Intend made to grant
Hath oken: there crown, something live, her in a thronest.

MISTRESS OVENEE:
I have made my father neither: we together not thence wi
